http://geologyandpython.com/dem-processing.html

In [4]:
from rasterio.transform import from_bounds, from_origin
from rasterio.warp import reproject, Resampling
import rasterio

In [6]:
dem_path= '/EXPLORE_temporary_datashare/EXPLORE-DC-dataset-examples/LROC/NAC/M101949648LE_lev2.tif'
dem_raster = rasterio.open('.' + dem_path)

In [13]:
west, south, east, north = bounds  = [-90.000672, 45.998852, -87.998534, 46.999431]
west, south, east, north = bounds  = west - .05, south - .05, east + .05, north + .05

In [22]:
src_shape

(50109, 6309)

In [14]:
src_crs = dem_raster.crs
src_shape = src_height, src_width = dem_raster.shape
src_transform = from_bounds(west, south, east, north, src_width, src_height)
source = dem_raster.read(1)

In [20]:
dst_crs = {'init': 'EPSG:9122'}
dst_transform = from_origin(268000.0, 5207000.0, 250, 250)
dem_array = np.zeros((451, 623))
dem_array[:] = np.nan

In [21]:
reproject(source,
          dem_array,
          src_transform=src_transform,
          src_crs=src_crs,
          dst_transform=dst_transform,
          dst_crs=dst_crs,
          resampling=Resampling.bilinear)

CRSError: The EPSG code is unknown. PROJ: proj_create_from_database: crs not found

In [ ]:
import os
import subprocess

import numpy as np
import rasterio
from rasterio import transform
from rasterio.warp import reproject, Resampling

tempdir = '/tmp'
tiffname = os.path.join(tempdir, 'example.tif')

with rasterio.Env():

    # Consider a 512 x 512 raster centered on 0 degrees E and 0 degrees N
    # with each pixel covering 15".
    rows, cols = src_shape = (512, 512)
    dpp = 1.0/240  # decimal degrees per pixel
    west, south, east, north = -cols*dpp/2, -rows*dpp/2, cols*dpp/2, rows*dpp/2
    src_transform = transform.from_bounds(west, south, east, north, cols, rows)
    src_crs = {'init': 'EPSG:4326'}
    source = np.ones(src_shape, np.uint8)*255

    # Prepare to reproject this rasters to a 1024 x 1024 dataset in
    # Web Mercator (EPSG:3857) with origin at -237481.5, 237536.4.
    dst_shape = (1024, 1024)
    dst_transform = transform.from_origin(-237481.5, 237536.4, 425.0, 425.0)
    dst_crs = {'init': 'EPSG:3857'}
    destination = np.zeros(dst_shape, np.uint8)

    reproject(
        source,
        destination,
        src_transform=src_transform,
        src_crs=src_crs,
        dst_transform=dst_transform,
        dst_crs=dst_crs,
        resampling=Resampling.nearest)

    # Assert that the destination is only partly filled.
    assert destination.any()
    assert not destination.all()

    # Write it out to a file.
    with rasterio.open(
            tiffname,
            'w',
            driver='GTiff',
            width=dst_shape[1],
            height=dst_shape[0],
            count=1,
            dtype=np.uint8,
            nodata=0,
            transform=dst_transform,
            crs=dst_crs) as dst:
        dst.write(destination, indexes=1)

info = subprocess.call(['open', tiffname])